# Week 2a: coincidence problem

In [ ]:
'''
Week 2a coincidence problem
'''
from numpy import random

def acquaintance():
    common = False;
    k = 0
    success = 0
    N = 10000
    population = 200000
    frA = 200
    frB = 200
    
    for i in range(N):
        common = False
        s = []
        for j in range(0, frB): # generate friends of B
            while True:
                k = random.randint(1, population)
                if k not in s:
                    break
            s = s + [k];
            ### BEGIN SOLUTION
            if k <= frA:
                common = True
                break
        if common:
            success = success + 1
        ### END SOLUTION
    print('Probability of common acquaintance = %g' % (success/N))
    
### MAIN ###
acquaintance()

# Week 6a: central limit theorem

In [1]:
### IMPORTS
from numpy import random


### Central limit function
def centrallimit():
    n = 10
    N = 30000
    sum = 0.0
#     file = open("data.txt", "w");
    results = []
    for i in range(N):
        sum = 0.0
        for j in range(n):
            if random.random() < 0.5:
                sum = sum + random.binomial(20, 0.1)
            else:
                sum = sum + random.binomial(20, 0.9);
        results = results + [sum]
    return results
#         file.write("\%g " % sum)
#     write(file, "\%g ", sum);
#     file.close()
    
# def results():
    
    
### MAIN ###
results = centrallimit()

In [12]:
from ..matplotlib import pyplot as plt

h = plot.hist(results, density = True, bins = 100)

ImportError: attempted relative import with no known parent package

# Week 6a: Confidence intervals for waiting time in single machine station

In [13]:
from PyCh import *
from statistics import mean

# =================================
# Tote definition
# =================================
@dataclass
class Job:
    entrytime: float = 0
    time_entered_buffer: float = 0

# =================================
# Generator definition
# =================================
@process
def Generator(env, sending_channel, ta_distribution):
    while True:
        x = Job()
        x.entrytime = env.now
        send = sending_channel.send(x)
        yield env.select(send)
        delay = ta_distribution()
        yield env.timeout(delay)

# =================================
# Buffer definition
# =================================
@process
def Buffer(env, receiving_channel, sending_channel):
    xs = [] # list of jobs
    while True:
        receiving = receiving_channel.receive() 
        sending = sending_channel.send(xs[0]) if len(xs)>0 else None
        yield env.select(sending,receiving)
        if selected(sending):
            waitingtime.append(env.now - xs[0].time_entered_buffer)
            xs = xs[1:]
        if selected(receiving):
            x = receiving.entity
            x.time_entered_buffer = env.now
            xs = xs + [x]

# =================================
# Machine definition
# =================================
@process
def Machine(env, receiving_channel, sending_channel, tp_distribution ):
    while True:
        receive = receiving_channel.receive()
        x = yield env.select(receive)
        
        delay = tp_distribution()
        yield env.timeout(delay)
        
        send = sending_channel.send(x)
        yield env.select(send)         

# =================================
# Exit definition
# =================================
@process
def Exit(env, receiving_channel, nJobs):
    total_flowtime = 0.0
    for j in range(nJobs):
        receive = receiving_channel.receive()
        x = yield env.select(receive)
        flowtime.append(env.now - x.entrytime);

# =================================
# GBME Model
# =================================
def GBME():
    env = Environment()

    ta_distribution = lambda: random.uniform(0.0, 2.0)
    tp_distribution = lambda: random.uniform(0.0, 1.0)
    nJobs = 1000

    a = Channel(env)
    b = Channel(env)
    c = Channel(env)

    G = Generator(env, a, ta_distribution)
    B = Buffer(env, a, b)
    M = Machine(env, b, c, tp_distribution)
    E = Exit(env, c, nJobs)

    env.run(until=E)
    
    mean_waitingtime = mean(waitingtime)
    mean_flowtime = mean(flowtime)
    
    return mean_waitingtime, mean_flowtime

# =================================
# Main
# =================================
for i in range(5):
    flowtime = []
    waitingtime = []
    mean_waitingtime, mean_flowtime = GBME()
    print("Mean waiting time in run %d = %g" % (i+1, mean_waitingtime) )
    

Mean waiting time in run 1 = 0.12313
Mean waiting time in run 2 = 0.13852
Mean waiting time in run 3 = 0.135822
Mean waiting time in run 4 = 0.171651
Mean waiting time in run 5 = 0.128013


# Week 6b: Serial production lines

In [ ]:
# =================================
# Exit definition
# =================================
@process
def Exit(env, receiving_channel, nJobs):
    total_flowtime = 0.0
    for j in range(nJobs):
        receive = receiving_channel.receive()
        x = yield env.select(receive)
        flowtime.append(env.now - x.entrytime);
        print(j, 'has exited the system')

# =================================
# Model
# =================================
def model():
    env = Environment()

    flowtime = []
    waitingtime = []

    a = [Channel(env) for c in range(3)] # a channel for each tier, each sending totes
    b = [Channel(env) for c in range(3)] # a channel for each tier, each sending totes

    G  = Generator(env, a[0], lambda: random.exponential(0.5))
    B1 = Buffer(   env, a[0], b[0])
    M1 = Machine(  env, b[0], a[1], lambda: 0.33)
    B2 = Buffer(   env, a[1], b[1])
    M2 = Machine(  env, b[1], a[2], lambda: random.uniform(0.0,0.8))
    E  = Exit(     env, a[2], 100000)

    env.run()

    print("Mean flowtime = %g" % mean(flowtime) )
    
# =================================
# Main
# =================================
model()

0 has exited the system
1 has exited the system
2 has exited the system
3 has exited the system
4 has exited the system
5 has exited the system
6 has exited the system
7 has exited the system
8 has exited the system
9 has exited the system
10 has exited the system
11 has exited the system
12 has exited the system
13 has exited the system
14 has exited the system
15 has exited the system
16 has exited the system
17 has exited the system
18 has exited the system
19 has exited the system
20 has exited the system
21 has exited the system
22 has exited the system
23 has exited the system
24 has exited the system
25 has exited the system
26 has exited the system
27 has exited the system
28 has exited the system
29 has exited the system
30 has exited the system
31 has exited the system
32 has exited the system
33 has exited the system
34 has exited the system
35 has exited the system
36 has exited the system
37 has exited the system
38 has exited the system
39 has exited the system
40 has exi

322 has exited the system
323 has exited the system
324 has exited the system
325 has exited the system
326 has exited the system
327 has exited the system
328 has exited the system
329 has exited the system
330 has exited the system
331 has exited the system
332 has exited the system
333 has exited the system
334 has exited the system
335 has exited the system
336 has exited the system
337 has exited the system
338 has exited the system
339 has exited the system
340 has exited the system
341 has exited the system
342 has exited the system
343 has exited the system
344 has exited the system
345 has exited the system
346 has exited the system
347 has exited the system
348 has exited the system
349 has exited the system
350 has exited the system
351 has exited the system
352 has exited the system
353 has exited the system
354 has exited the system
355 has exited the system
356 has exited the system
357 has exited the system
358 has exited the system
359 has exited the system
360 has exit

644 has exited the system
645 has exited the system
646 has exited the system
647 has exited the system
648 has exited the system
649 has exited the system
650 has exited the system
651 has exited the system
652 has exited the system
653 has exited the system
654 has exited the system
655 has exited the system
656 has exited the system
657 has exited the system
658 has exited the system
659 has exited the system
660 has exited the system
661 has exited the system
662 has exited the system
663 has exited the system
664 has exited the system
665 has exited the system
666 has exited the system
667 has exited the system
668 has exited the system
669 has exited the system
670 has exited the system
671 has exited the system
672 has exited the system
673 has exited the system
674 has exited the system
675 has exited the system
676 has exited the system
677 has exited the system
678 has exited the system
679 has exited the system
680 has exited the system
681 has exited the system
682 has exit

977 has exited the system
978 has exited the system
979 has exited the system
980 has exited the system
981 has exited the system
982 has exited the system
983 has exited the system
984 has exited the system
985 has exited the system
986 has exited the system
987 has exited the system
988 has exited the system
989 has exited the system
990 has exited the system
991 has exited the system
992 has exited the system
993 has exited the system
994 has exited the system
995 has exited the system
996 has exited the system
997 has exited the system
998 has exited the system
999 has exited the system
1000 has exited the system
1001 has exited the system
1002 has exited the system
1003 has exited the system
1004 has exited the system
1005 has exited the system
1006 has exited the system
1007 has exited the system
1008 has exited the system
1009 has exited the system
1010 has exited the system
1011 has exited the system
1012 has exited the system
1013 has exited the system
1014 has exited the syst

1285 has exited the system
1286 has exited the system
1287 has exited the system
1288 has exited the system
1289 has exited the system
1290 has exited the system
1291 has exited the system
1292 has exited the system
1293 has exited the system
1294 has exited the system
1295 has exited the system
1296 has exited the system
1297 has exited the system
1298 has exited the system
1299 has exited the system
1300 has exited the system
1301 has exited the system
1302 has exited the system
1303 has exited the system
1304 has exited the system
1305 has exited the system
1306 has exited the system
1307 has exited the system
1308 has exited the system
1309 has exited the system
1310 has exited the system
1311 has exited the system
1312 has exited the system
1313 has exited the system
1314 has exited the system
1315 has exited the system
1316 has exited the system
1317 has exited the system
1318 has exited the system
1319 has exited the system
1320 has exited the system
1321 has exited the system
1

...

98536 has exited the system
98537 has exited the system
98538 has exited the system
98539 has exited the system
98540 has exited the system
98541 has exited the system
98542 has exited the system
98543 has exited the system
98544 has exited the system
98545 has exited the system
98546 has exited the system
98547 has exited the system
98548 has exited the system
98549 has exited the system
98550 has exited the system
98551 has exited the system
98552 has exited the system
98553 has exited the system
98554 has exited the system
98555 has exited the system
98556 has exited the system
98557 has exited the system
98558 has exited the system
98559 has exited the system
98560 has exited the system
98561 has exited the system
98562 has exited the system
98563 has exited the system
98564 has exited the system
98565 has exited the system
98566 has exited the system
98567 has exited the system
98568 has exited the system
98569 has exited the system
98570 has exited the system
98571 has exited the

98841 has exited the system
98842 has exited the system
98843 has exited the system
98844 has exited the system
98845 has exited the system
98846 has exited the system
98847 has exited the system
98848 has exited the system
98849 has exited the system
98850 has exited the system
98851 has exited the system
98852 has exited the system
98853 has exited the system
98854 has exited the system
98855 has exited the system
98856 has exited the system
98857 has exited the system
98858 has exited the system
98859 has exited the system
98860 has exited the system
98861 has exited the system
98862 has exited the system
98863 has exited the system
98864 has exited the system
98865 has exited the system
98866 has exited the system
98867 has exited the system
98868 has exited the system
98869 has exited the system
98870 has exited the system
98871 has exited the system
98872 has exited the system
98873 has exited the system
98874 has exited the system
98875 has exited the system
98876 has exited the

99252 has exited the system
99253 has exited the system
99254 has exited the system
99255 has exited the system
99256 has exited the system
99257 has exited the system
99258 has exited the system
99259 has exited the system
99260 has exited the system
99261 has exited the system
99262 has exited the system
99263 has exited the system
99264 has exited the system
99265 has exited the system
99266 has exited the system
99267 has exited the system
99268 has exited the system
99269 has exited the system
99270 has exited the system
99271 has exited the system
99272 has exited the system
99273 has exited the system
99274 has exited the system
99275 has exited the system
99276 has exited the system
99277 has exited the system
99278 has exited the system
99279 has exited the system
99280 has exited the system
99281 has exited the system
99282 has exited the system
99283 has exited the system
99284 has exited the system
99285 has exited the system
99286 has exited the system
99287 has exited the

99550 has exited the system
99551 has exited the system
99552 has exited the system
99553 has exited the system
99554 has exited the system
99555 has exited the system
99556 has exited the system
99557 has exited the system
99558 has exited the system
99559 has exited the system
99560 has exited the system
99561 has exited the system
99562 has exited the system
99563 has exited the system
99564 has exited the system
99565 has exited the system
99566 has exited the system
99567 has exited the system
99568 has exited the system
99569 has exited the system
99570 has exited the system
99571 has exited the system
99572 has exited the system
99573 has exited the system
99574 has exited the system
99575 has exited the system
99576 has exited the system
99577 has exited the system
99578 has exited the system
99579 has exited the system
99580 has exited the system
99581 has exited the system
99582 has exited the system
99583 has exited the system
99584 has exited the system
99585 has exited the

99858 has exited the system
99859 has exited the system
99860 has exited the system
99861 has exited the system
99862 has exited the system
99863 has exited the system
99864 has exited the system
99865 has exited the system
99866 has exited the system
99867 has exited the system
99868 has exited the system
99869 has exited the system
99870 has exited the system
99871 has exited the system
99872 has exited the system
99873 has exited the system
99874 has exited the system
99875 has exited the system
99876 has exited the system
99877 has exited the system
99878 has exited the system
99879 has exited the system
99880 has exited the system
99881 has exited the system
99882 has exited the system
99883 has exited the system
99884 has exited the system
99885 has exited the system
99886 has exited the system
99887 has exited the system
99888 has exited the system
99889 has exited the system
99890 has exited the system
99891 has exited the system
99892 has exited the system
99893 has exited the